In [13]:
import numpy as np
import pandas as pd
import requests

import configparser
config = configparser.ConfigParser()
config.read('../../config.ini')

import geopandas as gpd
from shapely.geometry import LineString

In [22]:
df_magok_trips_peak = pd.read_json('../../data/bike_rental_history_weekday_community_peak.json')
df_magok_trips_peak = df_magok_trips_peak.loc[df_magok_trips_peak.loc[:,'origin_station_id'] != df_magok_trips_peak.loc[:,'desti_station_id'], :]
df_magok_trips_peak = df_magok_trips_peak.loc[(df_magok_trips_peak.loc[:,'return_minute_of_day'] - df_magok_trips_peak.loc[:,'rent_minute_of_day']>0) , :]
df_magok_trips_peak = df_magok_trips_peak.loc[df_magok_trips_peak.loc[:,'community']==0]

In [23]:
df_magok_trips_peak.head()

,birth_year,gender,user_type,origin_station_id,desti_station_id,rent_hour,return_hour,rent_minute_of_day,return_minute_of_day,community,origin_lat,origin_lng,origin_dist_to_subway,desti_lat,desti_lng,desti_dist_to_subway
0,1985,M,내국인,ST-2044,ST-2031,8,8,500,503,0,37.563927,126.830643,339.677010,37.566925,126.827438,19.851351
1,1985,M,내국인,ST-2044,ST-2031,8,8,497,500,0,37.563927,126.830643,339.677010,37.566925,126.827438,19.851351
2,1986,\N,내국인,ST-2058,ST-2031,8,8,481,490,0,37.550617,126.826881,858.806553,37.566925,126.827438,19.851351
3,1992,M,내국인,ST-2411,ST-2031,8,9,538,543,0,37.562092,126.820969,448.889646,37.566925,126.827438,19.851351
4,2000,F,내국인,ST-2411,ST-2031,8,8,529,534,0,37.562092,126.820969,448.889646,37.566925,126.827438,19.851351


In [25]:
df_magok_trips_peak.loc[:,'route'] = df_magok_trips_peak.apply(lambda x: f"{x['origin_station_id']}~{x['desti_station_id']}", axis=1)

In [7]:
url = "https://apis-navi.kakaomobility.com/v1/directions?origin={},{}&destination={},{}&avoid=motorway"
api_key = config['kakao']['key']
headers = {"Authorization": "KakaoAK " + api_key}
list_path = []
i=0
for route in df_magok_trips_peak.loc[:,'route'].unique().tolist():
    start_station, end_station = route.split('~')
    start_x, start_y = df_magok_trips_peak.loc[df_magok_trips_peak.loc[:,'origin_station_id']==start_station,['origin_lng', 'origin_lat']].values[0]
    end_x, end_y =df_magok_trips_peak.loc[df_magok_trips_peak.loc[:,'desti_station_id']==end_station,['desti_lng','desti_lat']].values[0]
    url_format = url.format(start_x, start_y, end_x, end_y)

    result = requests.get(url_format, headers=headers).json()
    try:
        nested_path = [i['vertexes'] for i in result['routes'][0]['sections'][0]['roads']]
        flatten_path = [coord for sublist in nested_path for coord in sublist]
        path = [[flatten_path[i], flatten_path[i+1]] for i in range(0, len(flatten_path), 2)]
    except:
        print(result)
        path = None
    list_path.append({'route':route, 'path':path})
    i=i+1
    if(i%100==0):
        print(i)
df_routes = pd.DataFrame(list_path)
df_routes = df_routes.loc[np.logical_not(df_routes.loc[:,'path'].isnull())]

100
200
300
400


In [28]:
df_magok_trips_peak_path = df_magok_trips_peak.merge(df_routes).loc[:,['origin_lat',
                                                                        'origin_lng',
                                                                        'desti_lat',
                                                                        'desti_lng',
                                                                        'route',
                                                                        'path',
                                                                        'rent_minute_of_day',
                                                                        'return_minute_of_day']]

list_timestamp = []
for i, row in df_magok_trips_peak_path.iterrows():
    start_time = row['rent_minute_of_day'] * 60
    end_time = row['return_minute_of_day'] * 60
    distance = LineString(row['path']).length
    velocity = distance / (end_time - start_time)
    path = row['path']
    timestamp = []
    timestamp.append(start_time)
    current_time = start_time
    for i in range(0,len(path)-2):
        segment_distance = LineString([path[i], path[i+1]]).length
        new_time = int(current_time + segment_distance/velocity)
        timestamp.append(new_time)
        current_time = new_time
    timestamp.append(end_time)
    list_timestamp.append(','.join([str(t) for t in timestamp]))

df_magok_trips_peak_path.loc[:,'timestamp'] = list_timestamp
df_magok_trips_peak_path.loc[:,'timestamp'] = df_magok_trips_peak_path.loc[:,'timestamp'].apply(lambda x: [int(n) for n in x.split(',')])

In [30]:
df_magok_trips_peak_path.to_json('../../data/magok_trips_peak_path.json', orient='records')

In [32]:
gdf_magok_trips_peak_path = gpd.GeoDataFrame(df_magok_trips_peak_path, geometry=[LineString(path) for path in df_magok_trips_peak_path.loc[:,'path']])

In [34]:
gdf_magok_trips_peak_path.drop(columns=['path','timestamp']).to_file('../../data/magok_trips_peak_path.geojson', driver='GeoJSON')